In [18]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D,Flatten,Dense,MaxPooling2D,Dropout

In [21]:
training_data=tf.keras.utils.image_dataset_from_directory(
    directory="N:\\deep learning project\\CNN\\emotion-detection-cnn\\dataset\\train",
    labels='inferred',
    label_mode='int',
    image_size=(48,48),
    batch_size=32,
)

Found 28709 files belonging to 7 classes.


In [22]:
testing_data=tf.keras.utils.image_dataset_from_directory(
    directory="N:\\deep learning project\\CNN\\emotion-detection-cnn\\dataset\\test",
    labels='inferred',
    label_mode='int',
    image_size=(48,48),
    batch_size=32,
)

Found 7178 files belonging to 7 classes.


train test split

In [23]:
def normalization(image,label):
    image=tf.cast(image/48.,tf.float32)
    return image,label

training_data=training_data.map(normalization)
testing_data=testing_data.map(normalization)
print(training_data)
print(testing_data)

<_MapDataset element_spec=(TensorSpec(shape=(None, 48, 48, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>
<_MapDataset element_spec=(TensorSpec(shape=(None, 48, 48, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


train test split

In [42]:
x_train=[]
x_test=[]
y_train=[]
y_test=[]
for images,labels in training_data:
    x_train.append(images)
    y_train.append(labels)
for images,labels in testing_data:
    x_test.append(images)
    y_train.append(labels)

In [43]:
x_train=tf.concat()

AttributeError: 'list' object has no attribute 'shape'

# Model Architecture(keras hyperparameter tuner API)

In [30]:
import keras_tuner as kt

In [38]:
def build_model(hp):
    """ tune the every hyperparameter of the cnn and fully connected layer using keras api"""
    model=Sequential()
    counter=0
    for i in range(hp.Int('no of layers',min_value=1,max_value=10)):
        if counter==0:
            model.add(Conv2D(
                hp.Int('no of filters_'+str(i),min_value=32,max_value=512,step=32),
                kernel_size=(3,3),
                activation=hp.Choice('activation function_'+str(i),values=['relu','selu','tanh','sigmoid','selu']),
                input_shape=(48,48,3)
                )
            )
            model.add(MaxPooling2D(pool_size=(2,2)))
            model.add(Dropout(hp.Choice('Dropout_'+str(i),values=[.1,.2,.3,.4,.5,.6,.7,.8,.9])))
            counter+=1
        else:
            model.add(Conv2D(
                hp.Int('no of filters_'+str(i),min_value=32,max_value=512,step=32),
                kernel_size=(3,3),
                activation=hp.Choice('activation function_'+str(i),values=['relu','selu','tanh','sigmoid','selu']),
            ))
            model.add(MaxPooling2D(pool_size=(2,2)))
            model.add(Dropout(hp.Choice('Dropout_'+str(i),values=[.1,.2,.3,.4,.5,.6,.7,.8,.9])))
    model.add(Flatten())
    # fully connected layers architecture
    for i in range(hp.Int('no of layers',min_value=1,max_value=10)):
        model.add(Dense(
            hp.Int('no of nodes_'+str(i),min_value=32,max_value=512,step=32),
            activation=hp.Choice('activation function_'+str(i),values=['relu','selu','sigmoid','tanh','elu'])
        ))
        model.add(Dropout(hp.Choice('Dropout_'+str(i),values=[.1,.2,.3,.4,.5,.6,.7,.8,.9])))
    model.add(Dense(7,activation='softmax'))
    # model compilation
    model.compile(
        optimizer=hp.Choice('best optimizer',values=['adam','rmsprop','adadelta','sgd','momentum']),
        loss='binary_crossentropy', # you can also give here the different loss function using hp.Choice() 
        metrics=['accuracy']
    )
    return model


In [39]:
tuner=kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5,project_name='bestOptimizer') #project_name making a folder containing the info

In [40]:
tuner.search(training_data,epoch=5,validation_data=testing_data)

Trial 2 Complete [00h 00m 01s]

Best val_accuracy So Far: None
Total elapsed time: 00h 00m 02s

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
8                 |3                 |no of layers
352               |32                |no of filters_0
relu              |selu              |activation function_0
0.5               |0.9               |Dropout_0
128               |384               |no of nodes_0
adam              |sgd               |best optimizer
256               |32                |no of filters_1
relu              |relu              |activation function_1
0.7               |0.1               |Dropout_1
192               |32                |no of filters_2
selu              |relu              |activation function_2
0.3               |0.1               |Dropout_2
448               |32                |no of nodes_1
448               |32                |no of nodes_2
352               |None              |no of filters_3
tanh              |None  

Traceback (most recent call last):
  File "n:\deep learning project\CNN\emotion-detection-cnn\env\lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "n:\deep learning project\CNN\emotion-detection-cnn\env\lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "n:\deep learning project\CNN\emotion-detection-cnn\env\lib\site-packages\keras_tuner\src\engine\tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "n:\deep learning project\CNN\emotion-detection-cnn\env\lib\site-packages\keras_tuner\src\engine\tuner.py", line 232, in _build_and_fit_model
    model = self._try_build(hp)
  File "n:\deep learning project\CNN\emotion-detection-cnn\env\lib\site-packages\keras_tuner\src\engine\tuner.py", line 164, in _try_bu

RuntimeError: Number of consecutive failures exceeded the limit of 3.
Traceback (most recent call last):
  File "n:\deep learning project\CNN\emotion-detection-cnn\env\lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "n:\deep learning project\CNN\emotion-detection-cnn\env\lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "n:\deep learning project\CNN\emotion-detection-cnn\env\lib\site-packages\keras_tuner\src\engine\tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "n:\deep learning project\CNN\emotion-detection-cnn\env\lib\site-packages\keras_tuner\src\engine\tuner.py", line 232, in _build_and_fit_model
    model = self._try_build(hp)
  File "n:\deep learning project\CNN\emotion-detection-cnn\env\li